In [ ]:
# need to put models in ./logs/{task}/{model}
# tasks = [banana_flowers, avocado_flowers]
# data folders: data (images) should be at ~/data/Rahan/{task}

task = 'banana_flowers'

# update the following command from via to coco if you want to load coco format data
data_types ={'banana_flowers':'via','avocado_flowers':'via','cucumber_flowers_3':'coco'}

In [2]:
from utils import *

/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from mrcnn.config import Config

class FlowersConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
        
    # Give the configuration a recognizable name
    NAME = task
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 1

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 512 # larger images for small flowers like avocado

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5

class InferenceConfig(FlowersConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
config = FlowersConfig()
inference_config = InferenceConfig()
# config.display()

# augmentations
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,iaa.GaussianBlur(sigma=(0, 0.5))),
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True)

In [8]:
# loading the model and data

f=Flower_detection(task) #avocado_flower
f.load_data(data_types[f.task])
f.model = f.load_model('last', 'inference') #load last model from designated folder

loading banana_flowers data
Images: 139
Classes: ['BG', 'flower']
Images: 44
Classes: ['BG', 'flower']
inference mode
load model from /home/gidish/cloned_libs/Mask_RCNN/logs/banana_flowers20180714T1529/mask_rcnn_banana_flowers_0025.h5
Re-starting from epoch 25


In [10]:
original_image, _, _, _, _ = modellib.load_image_gt(f.test_dataset, inference_config, 
                           image_id=0, use_mini_mask=False)

In [11]:
results = f.model.detect([original_image], verbose=1)
# Returns a list of dicts, one dict per image. The dict contains:
#         rois: [N, (y1, x1, y2, x2)] detection bounding boxes
#         class_ids: [N] int class IDs
#         scores: [N] float probability scores for the class IDs
#         masks: [H, W, N] instance binary masks

Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  512.00000  int64
anchors                  shape: (1, 65472, 4)         min:   -0.17712  max:    1.05188  float32
